In [1]:
def keep_indices_as_nonzero(arr, indices):
    arr = arr
    indices = np.asarray(indices, dtype=int)
    # 元の配列と同じ形状でゼロ配列を作成
    result = np.zeros_like(arr)
    # 指定されたインデックスに対応する要素をコピー
    result[:,indices] = arr[:,indices]
    return result


In [9]:
import numpy as np
arr = np.array([[-9.35256508e-01, -1.14798096e+00, -1.57383065e+00,  1.58821305e+00,
                 4.04343644e+00,  3.78885890e+00],
                [ 3.16103711e+00, -3.04401356e+00,  4.73352891e+00,  1.79542443e+00,
                 -2.49976973e+00,  3.20240891e+00]])

indices = [0, 1,2]  # 例: 2Dインデックス (浮動小数点でもOK)
indices = np.array(indices)  # インデックスをNumPy配列に変換

# 結果を表示
result = keep_indices_as_nonzero(arr, indices)
print(result)

[[-0.93525651 -1.14798096 -1.57383065  0.          0.          0.        ]
 [ 3.16103711 -3.04401356  4.73352891  0.          0.          0.        ]]


In [10]:
labels_ga = [[0,0,0],[1,1,1]]

In [11]:
labels_ga[0]

[0, 0, 0]

In [12]:
labels_ga[1]

[1, 1, 1]

In [14]:
labels_ga = np.array(labels_ga)
labels_ga.shape[0]

2

In [18]:
pop1 = [0  ,0,  0]
pop2 =   [3.5546141 ,  3.50958564 ,-9.79896805]

In [19]:
def Rosenbrock(x,n):
    value = 0
    for i in range(n-1):
        value += 100 * (x[i+1] - x[i]**2)**2 + (1 - x[i])**2
    return value

In [20]:
Rosenbrock(pop1,n=3)

2

In [21]:
pop = [ 2.66701798, -2.7367699   ,0.44133023 ,-4.50629121 , 2.76578144,  3.28390866]

In [22]:
Rosenbrock(pop,n=6)

49604.62904341413

In [ ]:
pop = [ 0.02686754  ,0.90678714 , 1.03429554 ,-0.98058751, -0.45037669 , 0.64408839]

In [24]:
Rosenbrock(pop,n=6)

732.7941650249563

In [25]:
global_min = [1,1,1,1,1,1]

In [26]:
Rosenbrock(global_min,n=6)

0

In [1]:
def Rosenbrock(x,n):
    value = 0
    for i in range(n-1):
        value += 100 * (x[i+1] - x[i]**2)**2 + (1 - x[i])**2
    return value

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import numpy as np
import random
from icecream import ic
from smt.problems import Rosenbrock
from rbf_surrogate_100_train import predict_surrogate
def Rosenbrock(x,n):
    value = 0
    for i in range(n-1):
        value += 100 * (x[i+1] - x[i]**2)**2 + (1 - x[i])**2
    return value

# シード値の設定
SEED = 42
np.random.seed(SEED)
random.seed(SEED)
# パラメータの設定
dim = 6

max_gen = 100
pop_size = 100
offspring_size = 100
bound_rastrigin = 5.12
bound = 5.12 # Typical bound for Rosenbrock function

# 初期集団の生成
def init_population(pop_size, dim, bound):
    return [np.random.uniform(-bound, bound, dim) for _ in range(pop_size)]

# 適合度の計算
def evaluate_population(population):
    return [Rosenbrock(individual,dim) for individual in population]
    # return objective_function_ga(population,dim)


    # eva = np.abs(predict_surrogate(population))
    # return eva

# ルーレット選択
def roulette_wheel_selection(population, fitness):
    # current_best_fitness_index = np.argmin(fitness)
    # return population[current_best_fitness_index]
    max_val = sum(fitness)
    pick = random.uniform(0, max_val)
    current = 0
 

    fitness_roulette = np.array(fitness)
    fitness_roulette= np.squeeze(fitness_roulette)
    for i, f in enumerate(fitness_roulette):
        current += f
        
        if current > pick.any():
            return population[i]
    return population[-1]

# UNDX交叉操作
def undx_crossover(parent1, parent2, parent3, dim):
    alpha = 0.5 #親の情報をどれだけ持ってくるか
    beta = 0.35 #乱数をどれだけ受け入れるか
    g = 0.5 * (parent1 + parent2)
    d = parent2 - parent1
    norm_d = np.linalg.norm(d)
    if norm_d == 0:#parent2とparent1が等しいとき
        return parent1, parent2
    d = d / norm_d#どれだけ解に近いか。
    
    rand = np.random.normal(0, 1, dim)
    

    child1 = g + alpha * (parent3 - g) + beta * np.dot(rand, d) * d #乱数
    child2 = g + alpha * (g - parent3) + beta * np.dot(rand, d) * d


    
    return child1, child2

# 変異操作
def mutate(individual, bound, mutation_rate=0.01):
    for i in range(len(individual)):
        if random.random() < mutation_rate:
            individual[i] = random.uniform(-bound, bound)
    return individual

# メインの遺伝的アルゴリズム
def genetic_algorithm(dim, max_gen, pop_size, offspring_size, bound):
    population = init_population(pop_size, dim, bound)

    #population=population[0]
    best_individual = None
    best_fitness = float('inf')
    fitness_history = []
    best_fitness_history = []
    avg_fitness_history = []    

            
    for generation in range(max_gen):
        population = np.array(population)

        fitness = evaluate_population(population)
        ic(fitness)
        current_best_fitness = min(fitness)
        avg_fitness = np.mean(fitness)
        best_fitness_history.append(current_best_fitness)
        avg_fitness_history.append(avg_fitness)
        if generation % 10 == 0:
            avg_fitness = np.mean(fitness)
            print(f"Generation {generation}: Best Fitness = {best_fitness}, Average Fitness = {avg_fitness}")

        fitness_history.append(np.mean(fitness))

        new_population = []
        while len(new_population) < offspring_size:
            parent1 = roulette_wheel_selection(population, fitness)
            parent2 = roulette_wheel_selection(population, fitness)
            parent3 = roulette_wheel_selection(population, fitness)
            child1, child2 = undx_crossover(parent1, parent2, parent3, dim)
            new_population.append(mutate(child1, bound))
            if len(new_population) < offspring_size:
                new_population.append(mutate(child2, bound))

        population = population + new_population
        # population = sorted(population, key=lambda x: predict_surrogate(population))[:pop_size]
        fitness = np.squeeze(fitness)
        current_best_fitness = min(fitness)

        if current_best_fitness < best_fitness:
            best_fitness = current_best_fitness
            index = np.argmin(fitness)
            best_individual = population[index]

            pop_surrogate = np.zeros((100,6))
            pop_surrogate[0] = best_individual
            tmp_fitness = predict_surrogate(pop_surrogate)


        # if abs(np.mean(fitness) - best_fitness) < 1e-6 and generation > 1000:
        #     break

    return best_individual, best_fitness, best_fitness_history, avg_fitness_history

# 実行
best_individual, best_fitness, best_fitness_history, avg_fitness_history = genetic_algorithm(dim, max_gen, pop_size, offspring_size, bound)
print(f"最良個体の適合度：{best_fitness}")
print(f"最良個体のパラメータ：{best_individual}")

ic| fitness: [np.float64(66323.31832670492),
              np.float64(91265.4130310264),
              np.float64(71716.87991346334),
              np.float64(32714.568755912067),
              np.float64(26050.43081275122),
              np.float64(115577.27803570186),
              np.float64(55550.51427266036),
              np.float64(80003.92863616088),
              np.float64(73074.67387659842),
              np.float64(172767.6290334915),
              np.float64(22234.4778265497),
              np.float64(110432.395290812),
              np.float64(72964.88973206276),
              np.float64(39966.06328000436),
              np.float64(30874.454820872514),
              np.float64(28168.41361009451),
              np.float64(169970.45710918232),
              np.float64(48907.773630638185),
              np.float64(97000.45953136907),
              np.float64(58674.994579147344),
              np.float64(84928.23430500749),
              np.float64(105532.07401932182),
      

Psi matrix: 
[[1.         0.10481672 0.14492793 ... 0.07048581 0.06608739 0.30442201]
 [0.10481672 1.         0.00995649 ... 0.00731931 0.00783275 0.09629405]
 [0.14492793 0.00995649 1.         ... 0.06467718 0.18186981 0.10068606]
 ...
 [0.07048581 0.00731931 0.06467718 ... 1.         0.01976919 0.12900385]
 [0.06608739 0.00783275 0.18186981 ... 0.01976919 1.         0.01597192]
 [0.30442201 0.09629405 0.10068606 ... 0.12900385 0.01597192 1.        ]]

Weights: [  36658.59098561   21269.98532644  -16624.95220807   39548.37907588
   29873.30033294   40309.68597799     630.34263983     267.08965386
   96668.26486794   32420.72364202   42781.68665404  -46958.49956756
   22322.37801434   25177.29463294   31639.2902233   111505.49804234
  -25394.12008043   74849.13865578  -17013.7184182    35787.09822713
  -28636.13552887   45475.92007335  -60824.49701474   41565.79470584
  -50339.07345262   38897.94035489   68653.10335967   84148.31752327
   49742.58780512  -47127.00098212    6745.3456976

              np.float64(29822.205418352896),
              np.float64(41590.48107147418),
              np.float64(206189.59608649477),
              np.float64(45443.947858121224),
              np.float64(18671.407157854563),
              np.float64(19026.196150762957),
              np.float64(127728.45793992729),
              np.float64(49212.15705938051),
              np.float64(196590.08890889297),
              np.float64(104731.05700970574),
              np.float64(42664.01331981281),
              np.float64(29906.739845487627),
              np.float64(78450.04603890122),
              np.float64(49357.78508315381),
              np.float64(52856.869391384185),
              np.float64(153410.65480135533),
              np.float64(5239.543635843683),
              np.float64(93495.56971332154),
              np.float64(82806.69302042245),
              np.float64(105389.80087943905),
              np.float64(145186.6094182145),
              np.float64(87867.77901819408)

Generation 0: Best Fitness = inf, Average Fitness = 78005.27504262463



              np.float64(640762.5388337857),
              np.float64(534504.479789784),
              np.float64(402921.44720859104),
              np.float64(129408.69915581188),
              np.float64(68149.48620624137),
              np.float64(390792.2301752195),
              np.float64(542284.2560701068),
              np.float64(918133.3254569952),
              np.float64(297733.062198195)]
ic| best_fitness: np.float64(1152.1393956364948)
ic| tmp_fitness[0]: array([41236.60228489])
ic| fitness: [np.float64(3206873.3827306833),
              np.float64(3715922.407914851),
              np.float64(522701.2068602428),
              np.float64(775941.6192885203),
              np.float64(1184849.317361592),
              np.float64(4630435.892585674),
              np.float64(2089391.98197378),
              np.float64(3304611.9153982354),
              np.float64(3711903.994347069),
              np.float64(404451.0511474673),
              np.float64(1555331.5169916924),
    

最良個体の適合度：1152.1393956364948
最良個体のパラメータ：[-3.97584112 -1.44632485  3.2630947   6.43841996  2.71400467 -3.34874391]
